In [1]:
train=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\loan\train_u6lujuX_CVtuZ9i.csv')



In [ ]:
train.describe().show()

In [2]:
test=spark.read.option('header','True').option('inferSchema','True').csv(r'C:\spark_projects\loan\test_Y3wMUE5_7gLdaTN.csv')
#test.count()

In [ ]:
train.schema

In [ ]:
from pyspark.sql.functions import col,sum
train.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in train.columns)).show()

In [ ]:
from pyspark.sql.functions import col,sum
test.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in test.columns)).show()

In [3]:
test=test.fillna({'Gender':'Male','Married':'Yes','Self_Employed':'No','Dependents':'0'})
train=train.fillna({'Gender':'Male','Married':'Yes','Self_Employed':'No','Dependents':'0'})

In [4]:
from pyspark.ml.feature import Imputer

test=test.withColumn('LoanD', test.LoanAmount.cast('float'))
test=test.withColumn('Loan_termD', test.Loan_Amount_Term.cast('float'))
test=test.withColumn('Credit_D', test.Credit_History.cast('float'))
imputer = Imputer(
    inputCols=["LoanD",'Loan_termD','Credit_D'], 
    outputCols=["Avg_imputed",'Amount_imp','Credit_History_imp']
)

test=imputer.fit(test).transform(test)

In [5]:
#train=train.dropna()
train=train.withColumn('LoanD', train.LoanAmount.cast('float'))
train=train.withColumn('Loan_termD', train.Loan_Amount_Term.cast('float'))
train=train.withColumn('Credit_D', train.Credit_History.cast('float'))
imputer = Imputer(
    inputCols=["LoanD",'Loan_termD','Credit_D'], 
    outputCols=["Avg_imputed",'Amount_imp','Credit_History_imp']
)

train=imputer.fit(train).transform(train)

In [16]:
from pyspark.sql.functions import *
train=train.withColumn('MM',(train.Gender=='M') & ( train.Married=='Yes'))
train=train.withColumn('MUM',(train.Gender=='M') & ( train.Married=='No'))
train=train.withColumn('FM',(train.Gender=='F') & ( train.Married=='Yes'))
train=train.withColumn('FUM',(train.Gender=='F') & ( train.Married=='No'))
train=train.withColumn('Int',((train.LoanAmount) / ( train.Loan_Amount_Term))*1000)
#train=train.withColumn('Ratio',((train.ApplicantIncome) / ( train.Int)))
#train.show(3)



In [7]:
train=train.withColumn('20k',(train.LoanAmount<=20000))
train=train.withColumn('50k',(train.LoanAmount>=20000)&(train.LoanAmount<=50000))
train=train.withColumn('>50k',(train.LoanAmount>=50000))

In [6]:
test=test.withColumn('MM',(test.Gender=='M') & ( test.Married=='Yes'))
test=test.withColumn('MUM',(test.Gender=='M') & ( test.Married=='No'))
test=test.withColumn('FM',(test.Gender=='F') & ( test.Married=='Yes'))
test=test.withColumn('FUM',(test.Gender=='F') & ( test.Married=='No'))
test=test.withColumn('Int',((test.Avg_imputed) / ( test.Amount_imp))*1000)


In [7]:
test=test.drop('LoanAmount')

test=test.drop('Loan_Amount_Term')

test=test.drop('Credit_History')
train=train.drop('LoanAmount')

train=train.drop('Loan_Amount_Term')

train=train.drop('Credit_History')

In [8]:
test=test.withColumnRenamed('Avg_imputed','LoanAmount')
test=test.withColumnRenamed('Amount_imp','Loan_Amount_Term')
test=test.withColumnRenamed('Credit_History_imp','Credit_History')

In [9]:
train=train.withColumnRenamed('Avg_imputed','LoanAmount')
train=train.withColumnRenamed('Amount_imp','Loan_Amount_Term')
train=train.withColumnRenamed('Credit_History_imp','Credit_History')

In [13]:
from pyspark.sql.functions import *
test=test.withColumn('TotalIncome',(test.ApplicantIncome) + ( test.CoapplicantIncome))
test=test.withColumn('LIncome',log(10.0,test.TotalIncome))
test=test.withColumn('LogofLAmount',log(10.0,test.LoanAmount))
train=train.withColumn('TotalIncome',(train.ApplicantIncome) + ( train.CoapplicantIncome))
train=train.withColumn('LIncome',log(10.0,train.TotalIncome))
train=train.withColumn('LogofLAmount',log(10.0,train.LoanAmount))

In [19]:
from pyspark.sql.functions import col,sum
train.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in train.columns)).show()

+-------+------+-------+----------+---------+-------------+---------------+-----------------+-------------+-----------+-----+----------+--------+----------+----------------+--------------+---+---+---+---+---+-----------+-------+------------+---+---+----+
|Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|Property_Area|Loan_Status|LoanD|Loan_termD|Credit_D|LoanAmount|Loan_Amount_Term|Credit_History| MM|MUM| FM|FUM|Int|TotalIncome|LIncome|LogofLAmount|20k|50k|>50k|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+-------------+-----------+-----+----------+--------+----------+----------------+--------------+---+---+---+---+---+-----------+-------+------------+---+---+----+
|      0|     0|      0|         0|        0|            0|              0|                0|            0|          0|   22|        14|      50|         0|               0|             0|  0|  0|  0|  0| 36|          0|      0|       

In [14]:
from pyspark.ml.feature import RFormula
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from pyspark.sql import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
categorical_columns = ['Dependents','Education','Self_Employed','Property_Area']
#categorical_columns = ['Dependents','Property_Area']

##=== build stages ======
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='stringindexed_' + c) for c in categorical_columns]
onehotencoder_stages = [OneHotEncoder(inputCol='stringindexed_' + c, outputCol='onehotencoded_' + c) for c in categorical_columns]
selected_columns = ['onehotencoded_' + c for c in categorical_columns] + ['MM','FM','FUM','MUM','TotalIncome','LoanAmount','Credit_History','Loan_Amount_Term']

vectorAssembler_stages = [VectorAssembler(inputCols=selected_columns, outputCol='features')]



In [17]:
all_stages = stringindexer_stages + onehotencoder_stages +vectorAssembler_stages
pipeline = Pipeline(stages=all_stages)

## fit pipeline model
final= pipeline.fit(train).transform(train)

In [18]:
testfinal= pipeline.fit(test).transform(test)

In [19]:
LoanGrant=StringIndexer(inputCol='Loan_Status', outputCol='Indexed')
finaltrain= LoanGrant.fit(final).transform(final)


In [20]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

evaluator = BinaryClassificationEvaluator(labelCol="Indexed")

In [21]:
lr=LogisticRegression(labelCol='Indexed',featuresCol='features')

paramGrid = ParamGridBuilder()\
  .addGrid(lr.maxIter, [10, 100])\
  .build()
    
crossval = CrossValidator(
    estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)


lrmodel = crossval.fit(finaltrain)

In [22]:
lrmodel.avgMetrics

[0.7707801781787584, 0.7580591644124155]

In [23]:
evaluator.evaluate(lrmodel.transform(finaltrain))

0.780855055292259

In [24]:
results=lrmodel.transform(testfinal)

In [25]:
results=results.withColumn('outputint',results.prediction.cast('int'))
results=results.withColumn('output',results.outputint.cast('string'))

converter = IndexToString(inputCol="outputint", outputCol="Loan_status",labels=['Y','N'])
converted = converter.transform(results)

In [26]:
from pyspark.sql.functions import col,sum
submission =converted.select(col("Loan_ID"),col("Loan_Status"))
submission.show(2)

+--------+-----------+
| Loan_ID|Loan_Status|
+--------+-----------+
|LP001015|          Y|
|LP001022|          Y|
+--------+-----------+
only showing top 2 rows



In [27]:
submission.write.csv('LoanPredictiontrial2.csv',header=True)

In [28]:
# bestPipeline = cvModel.bestModel
bestLRModel = lrmodel.bestModel
bestParams = bestLRModel.extractParamMap()

In [29]:
print(bestParams)

{Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2, Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.0, Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'): 'auto', Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='featuresCol', doc='features column name'): 'features', Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='fitIntercept', doc='whether to fit an intercept term'): True, Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='labelCol', doc='label column name'): 'Indexed', Param(parent='LogisticRegression_47c2b6ccd893a46a6f37', name='ma

In [71]:
rf = RandomForestClassifier(labelCol="Indexed")
 
paramGrid = ParamGridBuilder()\
    .addGrid(rf.maxDepth,[2,5])\
    .addGrid(rf.numTrees,[10,100])\
    .addGrid(rf.featureSubsetStrategy,['all','sqrt'])\
    .build()
    
crossval = CrossValidator(
    estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)


rfmodel = crossval.fit(finaltrain)
 #.addGrid(rf.featureSubsetStrategy,['sqrt')\

In [72]:
rfmodel.avgMetrics

[0.7448989033695762,
 0.7458992722675601,
 0.7586505532791029,
 0.7692338452611442,
 0.7607810695367393,
 0.7468852537011978,
 0.7687772208469259,
 0.7682786790003096]

In [73]:
evaluator.evaluate(rfmodel.transform(finaltrain))

0.8288593503159565

In [74]:
gbt = GBTClassifier(labelCol="Indexed",featuresCol="features")

paramGrid = ParamGridBuilder()\
    .addGrid(gbt.maxDepth,[2,5])\
    .addGrid(gbt.maxIter,[10,100])\
    .build()
    
crossval = CrossValidator(
    estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=6)


bgtmodel = crossval.fit(finaltrain)

In [75]:
bgtmodel.avgMetrics

[0.7545799853927881,
 0.7557201477043854,
 0.7344869975918898,
 0.7482788278782198]

In [ ]:
import pyspark.ml.evaluation as ev
evaluator = ev.BinaryClassificationEvaluator(
    rawPredictionCol='probability', 
    labelCol='Indexed')

In [ ]:
print(evaluator.evaluate(rfpredictions, 
    {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(rfpredictions, 
   {evaluator.metricName: 'areaUnderPR'}))

In [ ]:
rftestpredictions=rftestpredictions.withColumn('outputint',rftestpredictions.prediction.cast('int'))
rftestpredictions=rftestpredictions.withColumn('output',rftestpredictions.outputint.cast('string'))
rftestpredictions.show(2)


In [ ]:
result=result.withColumn('outputint',result.prediction.cast('int'))
result=result.withColumn('output',result.outputint.cast('string'))


converter = IndexToString(inputCol="outputint", outputCol="Loan_status",labels=['Y','N'])
converted = converter.transform(result)

In [ ]:
converted.show(2)


In [ ]:
from pyspark.sql.functions import col,sum
submission =converted.select(col("Loan_ID"),col("Loan_Status"))
submission.show(2)


In [ ]:
submission.count()

In [ ]:
submission.write.csv('LoanPrediction.csv',header=True)

In [ ]:
# display CV score
auc_roc = model.avgMetrics[0]
print("AUC ROC = %g" % auc_roc)
gini = (2 * auc_roc - 1)
print("GINI ~=%g" % gini)